# Imports

In [56]:
import pandas as pd
import numpy as np
import json
import datetime
import ast
import seaborn as sns
import dtale
import ast
import pickle
import category_encoders as ce

from sklearn                 import preprocessing as pp
from sklearn                 import model_selection as ms

from lightgbm                import LGBMClassifier
from sklearn.impute          import KNNImputer
from sklearn.ensemble        import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from xgboost                 import XGBRegressor
from sklearn.linear_model    import LogisticRegression, SGDRegressor
from sklearn.neighbors       import KNeighborsRegressor
from sklearn.tree            import DecisionTreeRegressor
from sklearn.ensemble        import RandomForestRegressor, BaggingRegressor
from sklearn.metrics         import accuracy_score, recall_score, precision_score, balanced_accuracy_score, f1_score, roc_curve, confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes     import GaussianNB
from lightgbm                import LGBMRegressor

## Helper Functions

In [57]:
def test_preparation(X_test, y_test):
    X_test = rescalling(X_test)
    X_test = transform(X_test)
    y_test = log_var_resposta(y_test)
    
    return X_test, y_test


def rescalling (X_test):
    
    mm = pickle.load(open('parameters/mm_avg_delivery_time_days.pkl','rb'))
    X_test['avg_delivery_time_days'] = mm.transform(X_test[['avg_delivery_time_days']])
    
    rs_ar = pickle.load(open('parameters/rs_average_rating.pkl','rb'))
    
    X_test['average_rating'] = rs_ar.transform(X_test[['average_rating']])
    
    rs_nr = pickle.load(open('parameters/rs_number_of_reviews.pkl','rb'))
    
    X_test['number_of_reviews'] = rs_nr.transform(X_test[['number_of_reviews']])
    
    return X_test

def transform (df):
    
    for att in ['Ideal For', 'Reversible', 'Suitable For', 'seller', 'product', 'Type', 'Sleeve', 'Fit', 'Fabric']:
        te = pickle.load(open(f'parameters/te_{att}.pkl','rb'))
        df = te.transform(df)        
    
    for att in ['brand','Style Code','Pack of', 'Pattern']:
        fe = pickle.load(open(f'parameters/fe_{att}.pkl', 'rb'))
        df = fe.transform(df)
    
    return df

def log_var_resposta(df):
    df = np.log1p(df)
    return df

def limpeza (df):
    df = df.drop(columns=['_id', 'pid','description', 'images'])
    # transformando de string para data
    df['crawled_at'] = pd.to_datetime(df['crawled_at']).dt.date

    return df

def feature_engen(df):
    ## out_of_stock - transformar para int 
    df['out_of_stock'] = df['out_of_stock'].astype('int64')

    # brand - substituir por outros
    df['brand'] = df['brand'].apply(lambda x: x.lower())

    # criando feature product
    df['product'] = df[['category', 'sub_category']].apply(lambda x: x['category'] + '_' + x['sub_category'], axis = 1)
    
    df.loc[df['number_of_reviews'] == 0, 'average_rating'] = -1
    
    import ast

    # pega string de dict e cria uma lista de dicionários
    def f(x):
        try:
            return ast.literal_eval(str(x))   
        except Exception as e:
            print(e)
            return []



    def junta_dict(dict_list):
        dicionario = {}
        for d in dict_list:
            for key, value in d.items():
                dicionario[key] = value
        return dicionario

    list = df['product_details'].apply(lambda x: f(x))

    df_details = pd.DataFrame([junta_dict(row) for row in list], index = list.index)

    df_drop_columns = df_details.drop(columns=['', ' '])

    # Removendo valores faltantes acima de 50%
    limite_nulos = len(df_details) * 0.5  
    df_drop = df_details.dropna(thresh = limite_nulos, axis=1)
    df_new = df_drop.copy()

    # Trocar os NaN por 'outros'

    df_new.loc[df_new['Fabric'].isnull(), 'Fabric'] = 'outros'
    df_new.loc[df_new['Pattern'].isnull(), 'Pattern'] = 'outros'
    df_new.loc[df_new['Style Code'].isnull(), 'Style Code'] = 'outros'
    df_new.loc[df_new['Pack of'].isnull(), 'Pack of'] = 'outros'
    df_new.loc[df_new['Type'].isnull(), 'Type'] = 'outros'
    df_new.loc[df_new['Sleeve'].isnull(), 'Sleeve'] = 'outros'
    df_new.loc[df_new['Fit'].isnull(), 'Fit'] = 'outros'
    df_new.loc[df_new['Ideal For'].isnull(), 'Ideal For'] = 'outros'
    df_new.loc[df_new['Suitable For'].isnull(), 'Suitable For'] = 'outros'
    df_new.loc[df_new['Reversible'].isnull(), 'Reversible'] = 'outros'
    df_new.loc[df_new['Fabric Care'].isnull(), 'Fabric Care'] = 'outros'

    df_new.head()

    def type_sex(value):
        if value == 'Men':
            return 'Men'
        elif value == 'Men, Boys':
            return 'all_ages'
        elif value == 'Boys, Men':
            return 'all_ages'
        elif value == 'Boys, Girls, Men, Women':
            return 'unisex_all_ages'
        elif value == 'Women, Men':
            return 'Unisex'
        else:
            return 'outros'

    df_new['Ideal For'] = df_new['Ideal For'].map(type_sex)

    df = pd.concat([df, df_new], axis=1)
    
    df['cotton'] = df['Fabric'].apply(lambda x: 1 if 'cotton' in x.lower() else 0)
    df['poly'] = df['Fabric'].apply(lambda x: 1 if 'poly' in x.lower() else 0)
    df['not_cotton_or_poly'] = df['Fabric'].apply(lambda x: 1 if ('poly' not in x.lower()) and ('cotton' not in x.lower()) else 0)
    
    return df

# Load Data

In [58]:
# Opening JSON file
train = open('dataset/raw/train.json')
 
# returns JSON object as 
# a dictionary
data_train_aux = json.load(train)

data_train = pd.json_normalize(data_train_aux, record_path = 'data')
data_train.columns = data_train_aux['columns']
data_train.head()

,_id,average_rating,number_of_reviews,brand,category,crawled_at,description,images,out_of_stock,avg_delivery_time_days,pid,product_details,seller,sub_category,fabrication_time,title,actual_price
0,53df9662-e500-569c-946e-0c8d215a72cd,3.2,26,East I,Clothing and Accessories,2021-02-10 21:17:28,Navy Blue Printed Boxers Has An Inner Elasti...,['https://rukminim1.flixcart.com/image/128/128...,False,8,BXRFTZF7JGX75DAW,"[{'Color': 'Dark Blue'}, {'Fabric': 'Pure Cott...",ZIYAA,Innerwear and Swimwear,653,Printed Men Boxer (Pack of 1),849.0
1,d0142842-84f7-537d-a06f-d85b76488a5f,4.0,33,dream o,Clothing and Accessories,2021-02-11 01:02:46,smiley printed tshirt on round neck cotton tshirt,['https://rukminim1.flixcart.com/image/128/128...,False,12,TSHFWQM96UHR6A4Q,"[{'Type': 'Round Neck'}, {'Sleeve': 'Short Sle...",Dream Onn Creations,Topwear,668,Printed Men Round Neck Orange T-Shirt,699.0
2,79c8f0d7-30b1-5dd4-9f2f-2fe97782b027,3.9,32,Free Authori,Clothing and Accessories,2021-02-11 00:43:37,Free Authority Presents this Crew Neck Yellow ...,['https://rukminim1.flixcart.com/image/128/128...,False,11,SWSFWCXH2WF6ZYRB,"[{'Color': 'Yellow'}, {'Fabric': 'Polycotton'}...",BioworldMerchandising,Winter Wear,53,Full Sleeve Graphic Print Men Sweatshirt,1499.0
3,0531c28c-7c50-5fbd-9ce3-a7cae3243ad5,3.8,31,HUMBE,Clothing and Accessories,2021-02-10 21:22:10,Cotton Blend FabricCollar / Polo Neck White & ...,['https://rukminim1.flixcart.com/image/128/128...,False,11,TSHFHQH3HKDAGGK9,"[{'Type': 'Polo Neck'}, {'Sleeve': 'Short Slee...",HUMBERT,Topwear,510,"Solid Men Polo Neck Light Blue, White T-Shirt ...",1699.0
4,d604baad-472e-5c18-86a3-7b46d4a890c2,2.4,20,Rose We,Clothing and Accessories,2021-02-10 23:36:36,undefined,['https://rukminim1.flixcart.com/image/128/128...,False,5,TSHFW9CJZSYUU6UX,"[{'Type': 'Round Neck'}, {'Sleeve': 'Short Sle...",Rupalcollectionjaipur,Topwear,496,Printed Men Round Neck White T-Shirt,599.0


# Data Description

In [59]:
# retirar descrição
# retirar imagem
# verificar ids duplicados em id produto
# Verificar outliers no price

## Removendo colunas

In [60]:
data_train = data_train.drop(columns=['description', 'images'])

## Transformando tipo de dados

In [61]:
data_train.dtypes

_id                        object
average_rating            float64
number_of_reviews           int64
brand                      object
category                   object
crawled_at                 object
out_of_stock                 bool
avg_delivery_time_days      int64
pid                        object
product_details            object
seller                     object
sub_category               object
fabrication_time            int64
title                      object
actual_price              float64
dtype: object

In [62]:
# transformando de string para data
data_train['crawled_at'] = pd.to_datetime(data_train['crawled_at']).dt.date

## Tratando valores nulos

In [63]:
data_train.isna().sum()/data_train.shape[0]

_id                       0.000000
average_rating            0.000000
number_of_reviews         0.000000
brand                     0.000000
category                  0.000000
crawled_at                0.000000
out_of_stock              0.000000
avg_delivery_time_days    0.000000
pid                       0.000000
product_details           0.000000
seller                    0.000000
sub_category              0.000000
fabrication_time          0.000000
title                     0.000000
actual_price              0.149985
dtype: float64

In [64]:
## Tratando variaveis respostas nulas
prices = data_train[['seller','pid', 'actual_price']].groupby('pid').max().reset_index()

precos_nulos = data_train.loc[data_train['actual_price'].isna(), ['_id','pid']]

precos_recuperados = pd.merge(precos_nulos, prices, how = 'inner', on = 'pid')

data_train_aux = pd.merge(data_train, precos_recuperados[['_id', 'actual_price']], on = '_id', how = 'left')
data_train_aux['actual_price'] = data_train_aux[['actual_price_x', 'actual_price_y']].apply(lambda x: x['actual_price_x'] if x['actual_price_x'] > 0 else x['actual_price_y'], axis=1)
data_train['actual_price'] = data_train_aux['actual_price']

# data_train = data_train.dropna(subset=['actual_price'])

# data_train = data_train.loc[data_train['actual_price'] <= 8000]

In [65]:
data_train.isna().sum()/data_train.shape[0]

_id                       0.000000
average_rating            0.000000
number_of_reviews         0.000000
brand                     0.000000
category                  0.000000
crawled_at                0.000000
out_of_stock              0.000000
avg_delivery_time_days    0.000000
pid                       0.000000
product_details           0.000000
seller                    0.000000
sub_category              0.000000
fabrication_time          0.000000
title                     0.000000
actual_price              0.139002
dtype: float64

## Entendendo product Details

In [66]:
import ast

# pega string de dict e cria uma lista de dicionários
def f(x):
    try:
        return ast.literal_eval(str(x))   
    except Exception as e:
        print(e)
        return []

teste = data_train['product_details'].apply(lambda x: f(x))

def junta_dict(dict_list):
    dicionario = {}
    for d in dict_list:
        for key, value in d.items():
            dicionario[key] = value
    return dicionario


df = pd.DataFrame([junta_dict(row) for row in teste])

In [67]:
df = pd.DataFrame([junta_dict(row) for row in teste])
df.isna().sum().sort_values()

Fabric            1733
Style Code        1939
Pattern           1948
Fabric Care       5015
Suitable For      7168
                 ...  
Width at Base    23307
Pleated          23308
Design           23308
Fabric care      23308
Foot Coverage    23308
Length: 125, dtype: int64

# Feature Engineering

In [68]:
df4 = data_train.copy()

In [69]:
## out_of_stock - transformar para int 
df4['out_of_stock'] = df4['out_of_stock'].astype('int64')

# brand - substituir por outros
df4['brand'] = df4['brand'].apply(lambda x: x.lower())

# criando feature product
df4['product'] = df4[['category', 'sub_category']].apply(lambda x: x['category'] + '_' + x['sub_category'], axis = 1)

df4.loc[df4['number_of_reviews'] == 0, 'average_rating'] = -1

In [70]:
# pega string de dict e cria uma lista de dicionários
def f(x):
    try:
        return ast.literal_eval(str(x))   
    except Exception as e:
        print(e)
        return []



def junta_dict(dict_list):
    dicionario = {}
    for d in dict_list:
        for key, value in d.items():
            dicionario[key] = value
    return dicionario

list = df4['product_details'].apply(lambda x: f(x))

df_details = pd.DataFrame([junta_dict(row) for row in list], index = list.index)

df_drop_columns = df_details.drop(columns=['', ' '])

# Removendo valores faltantes acima de 50%
limite_nulos = len(df_details) * 0.5  
df_drop = df_details.dropna(thresh = limite_nulos, axis=1)
df_new = df_drop.copy()

# Trocar os NaN por 'outros'

df_new.loc[df_new['Fabric'].isnull(), 'Fabric'] = 'outros'
df_new.loc[df_new['Pattern'].isnull(), 'Pattern'] = 'outros'
df_new.loc[df_new['Style Code'].isnull(), 'Style Code'] = 'outros'
df_new.loc[df_new['Pack of'].isnull(), 'Pack of'] = 'outros'
df_new.loc[df_new['Type'].isnull(), 'Type'] = 'outros'
df_new.loc[df_new['Sleeve'].isnull(), 'Sleeve'] = 'outros'
df_new.loc[df_new['Fit'].isnull(), 'Fit'] = 'outros'
df_new.loc[df_new['Ideal For'].isnull(), 'Ideal For'] = 'outros'
df_new.loc[df_new['Suitable For'].isnull(), 'Suitable For'] = 'outros'
df_new.loc[df_new['Reversible'].isnull(), 'Reversible'] = 'outros'
df_new.loc[df_new['Fabric Care'].isnull(), 'Fabric Care'] = 'outros'

df_new.head()

def type_sex(value):
    if value == 'Men':
        return 'Men'
    elif value == 'Men, Boys':
        return 'all_ages'
    elif value == 'Boys, Men':
        return 'all_ages'
    elif value == 'Boys, Girls, Men, Women':
        return 'unisex_all_ages'
    elif value == 'Women, Men':
        return 'Unisex'
    else:
        return 'outros'

df_new['Ideal For'] = df_new['Ideal For'].map(type_sex)

df4 = pd.concat([df4, df_new], axis=1)

# Novas Features de roupa

df4['cotton'] = df4['Fabric'].apply(lambda x: 1 if 'cotton' in x.lower() else 0)
df4['poly'] = df4['Fabric'].apply(lambda x: 1 if 'poly' in x.lower() else 0)
df4['not_cotton_or_poly'] = df4['Fabric'].apply(lambda x: 1 if ('poly' not in x.lower()) and ('cotton' not in x.lower()) else 0)

In [71]:
df4.isna().sum()

_id                          0
average_rating               0
number_of_reviews            0
brand                        0
category                     0
crawled_at                   0
out_of_stock                 0
avg_delivery_time_days       0
pid                          0
product_details              0
seller                       0
sub_category                 0
fabrication_time             0
title                        0
actual_price              3240
product                      0
Fabric                       0
Pattern                      0
Style Code                   0
Pack of                      0
Type                         0
Sleeve                       0
Fit                          0
Ideal For                    0
Suitable For                 0
Reversible                   0
Fabric Care                  0
cotton                       0
poly                         0
not_cotton_or_poly           0
dtype: int64

# EDA

In [72]:
# # Assigning a reference to a running D-Tale process
# d = dtale.show(df4)

# # Accessing data associated with D-Tale process
# tmp = d.data.copy()
# tmp['d'] = 4
# d.data = tmp

# # Shutting down D-Tale process
# #d.kill()
# d.open_browser()

# # There is also some helpful metadata about the process
# d._data_id  # the process's data identifier
# d._url  # the url to access the process
# d2 = dtale.get_instance(d._data_id)  # returns a new reference to the instance running at that data_id
# dtale.instances()  # prints a list of all ids & urls of running D-Tale sessions

# Data Preparation

In [73]:
df6 = df4.copy()

In [74]:
df6.isna().sum()

_id                          0
average_rating               0
number_of_reviews            0
brand                        0
category                     0
crawled_at                   0
out_of_stock                 0
avg_delivery_time_days       0
pid                          0
product_details              0
seller                       0
sub_category                 0
fabrication_time             0
title                        0
actual_price              3240
product                      0
Fabric                       0
Pattern                      0
Style Code                   0
Pack of                      0
Type                         0
Sleeve                       0
Fit                          0
Ideal For                    0
Suitable For                 0
Reversible                   0
Fabric Care                  0
cotton                       0
poly                         0
not_cotton_or_poly           0
dtype: int64

## Split Dataset

In [75]:
X = df6.drop(['_id','pid','actual_price'], axis=1)
y = df6['actual_price']

X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.01, random_state=1)
data_train = pd.concat([X_train, y_train], axis=1)
data_test = pd.concat([X_test, y_test], axis=1)

## Rescalling

In [76]:
X_train.columns

Index(['average_rating', 'number_of_reviews', 'brand', 'category',
       'crawled_at', 'out_of_stock', 'avg_delivery_time_days',
       'product_details', 'seller', 'sub_category', 'fabrication_time',
       'title', 'product', 'Fabric', 'Pattern', 'Style Code', 'Pack of',
       'Type', 'Sleeve', 'Fit', 'Ideal For', 'Suitable For', 'Reversible',
       'Fabric Care', 'cotton', 'poly', 'not_cotton_or_poly'],
      dtype='object')

### Min Max Scaler

In [77]:
mm = pp.MinMaxScaler()

X_train['avg_delivery_time_days'] = mm.fit_transform(X_train[['avg_delivery_time_days']])
pickle.dump(mm, open('parameters/mm_avg_delivery_time_days.pkl','wb'))

### Robust Scaler

In [78]:
rs_average_rating   = pp.RobustScaler()
rs_number_of_reviews = pp.RobustScaler()

X_train['average_rating'] = rs_average_rating.fit_transform(X_train[['average_rating']])
pickle.dump(rs_average_rating, open('parameters/rs_average_rating.pkl','wb'))

X_train['number_of_reviews'] = rs_number_of_reviews.fit_transform(X_train[['number_of_reviews']])
pickle.dump(rs_number_of_reviews, open('parameters/rs_number_of_reviews.pkl','wb'))

## Transformation

### One Hot Enconde

In [79]:
# 'out_of_stock'

### Target Encode

In [80]:
for att in ['Ideal For', 'Reversible', 'Suitable For', 'seller', 'product', 'Type', 'Fit', 'Sleeve', 'Fabric']:
    te = ce.TargetEncoder(cols=[att]).fit(X_train, y_train)
    pickle.dump(te, open(f'parameters/te_{att}.pkl','wb'))
    X_train = te.transform(X_train)

### Frequency Encode

In [81]:
for att in ['brand','Style Code','Pack of', 'Pattern']:
    fe =  ce.CountEncoder(cols=[att], normalize=True).fit(X_train)
    pickle.dump(fe,open(f'parameters/fe_{att}.pkl', 'wb'))
    X_train = fe.transform(X_train)

### Response Variable

In [82]:
y_train = np.log1p(y_train)

In [83]:
data_train = pd.concat([X_train, y_train], axis =1)

## Test Preparation

In [84]:
X_test_mod, y_test_mod = test_preparation(X_test, y_test)

data_test = pd.concat([X_test_mod, y_test_mod])

# Convert feature names to strings
data_test.columns = data_test.columns.astype(str)

# Apply KNNImputer
imputer = KNNImputer(n_neighbors=5)
data_test = pd.DataFrame(imputer.fit_transform(data_test.select_dtypes(include=['int64', 'float64'])), columns=data_test.select_dtypes(include=['int64', 'float64']).columns)

# X_test_mod = data_test.drop(columns=0)  
# y_test_mod = data_test[0]


data_train = pd.concat([X_train, y_train], axis =1)

data_train.columns = data_train.columns.astype(str)

# Apply KNNImputer
imputer = KNNImputer(n_neighbors=5)
data_train = pd.DataFrame(imputer.fit_transform(data_train.select_dtypes(include=['int64', 'float64'])), columns=data_train.select_dtypes(include=['int64', 'float64']).columns)

X_train = data_train.drop(columns='actual_price')  
y_train = data_train['actual_price']



# Feature Selection

In [85]:
X_train.columns.tolist()

['average_rating',
 'number_of_reviews',
 'brand',
 'out_of_stock',
 'avg_delivery_time_days',
 'seller',
 'fabrication_time',
 'product',
 'Fabric',
 'Pattern',
 'Style Code',
 'Pack of',
 'Type',
 'Sleeve',
 'Fit',
 'Ideal For',
 'Suitable For',
 'Reversible',
 'cotton',
 'poly',
 'not_cotton_or_poly']

In [86]:
features_selected = [
'average_rating',
'number_of_reviews',
'brand',
# 'category',
# 'crawled_at',
'out_of_stock',
'avg_delivery_time_days',
#  'product_details',
'seller',
#  'sub_category',
#  'fabrication_time',
#  'title',
 'Fabric',
 'Pattern',
'cotton',
'poly',
'not_cotton_or_poly',
'Style Code',
'Pack of',
'Type',
 'Sleeve',
 'Fit',
'Ideal For',
'Suitable For',
'Reversible',
#  'Fabric Care',
'product'
]

## Importancia das árvores

In [87]:
# Crie e treine o modelo LGBMClassifier
lgb_model = LGBMRegressor(n_jobs=-1, random_state=42)
X_train_sel = X_train[features_selected]
lgb_model.fit(X_train_sel, y_train)

# Obtenha as importâncias das características do modelo
importances = lgb_model.feature_importances_

# Ordene as características por importância decrescente
indices = np.argsort(importances)[::-1]

# Imprima o ranking das características
print('Feature ranking')
df = pd.DataFrame()

for i, j in zip(X_train_sel.columns, importances):
    aux = pd.DataFrame({'feature': i, 'importance': j}, index=[0])
    df = pd.concat([df, aux], axis=0)

print(df.sort_values('importance', ascending=False))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001453 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 821
[LightGBM] [Info] Number of data points in the train set: 23075, number of used features: 20
[LightGBM] [Info] Start training from score 7.141837


Feature ranking
                  feature  importance
0                   brand         589
0                  seller         520
0                    Type         353
0          average_rating         264
0                  Fabric         257
0                 Pack of         199
0                 product         159
0                 Pattern         132
0              Style Code         124
0                  Sleeve         104
0              Reversible          69
0                     Fit          65
0            out_of_stock          38
0               Ideal For          35
0                    poly          31
0            Suitable For          31
0                  cotton          16
0      not_cotton_or_poly          14
0       number_of_reviews           0
0  avg_delivery_time_days           0


# Machine Learning

In [88]:
def MachineLearning(X_train, y_train, X_test, y_test, features_selected):
    SEED = 42

    lista_de_medidas = ['SMAPE']
    nome_das_medidas = ['SMAPE']

    lista_de_modelos = [XGBRegressor(learning_rate=0.1, n_estimators=50, max_depth=3),
                        LGBMRegressor(random_state=SEED, n_jobs=-1, force_row_wise=True, ),
                        DecisionTreeRegressor(random_state=SEED),
                        KNeighborsRegressor(n_neighbors=5,  weights='distance',n_jobs=-1),
                        BaggingRegressor(),
                        RandomForestRegressor(random_state=SEED)]

    nome_dos_modelos = ['XGBoost', 
                        'LGBM', 
                        'DecisionTree',
                        'KNN',
                        'Bagging',
                        'RandomForest']

    resultados0 = {}
    
    X_train_selected = X_train[features_selected]
    X_test_selected = X_test[features_selected]


    for i in range(len(lista_de_modelos)):
        print('Rodando modelo: ' + nome_dos_modelos[i])
        
        model = lista_de_modelos[i]
        
        model.fit(X_train_selected, y_train)
        
        y_pred = model.predict(X_test_selected)
        
        smape = 100 / len(y_test) * np.sum(2 * np.abs(np.expm1(y_pred) - np.expm1(y_test)) / (np.abs(np.expm1(y_test)) + np.abs(np.expm1(y_pred))))

        resultados0[nome_dos_modelos[i]] = [smape]
    
    resultados = pd.DataFrame(resultados0, index = nome_das_medidas).T

    return resultados

In [89]:
MachineLearning(X_train, y_train, X_test_mod, y_test_mod, features_selected)

Rodando modelo: XGBoost
Rodando modelo: LGBM
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 821
[LightGBM] [Info] Number of data points in the train set: 23075, number of used features: 20
[LightGBM] [Info] Start training from score 7.141837


Rodando modelo: DecisionTree
Rodando modelo: KNN
Rodando modelo: Bagging
Rodando modelo: RandomForest


,SMAPE
XGBoost,19.313168
LGBM,13.772792
DecisionTree,11.102171
KNN,9.345359
Bagging,10.080063
RandomForest,10.004798


# Fine Tuning

- Não possui cross validation
- Não realiza reencode das variáveis
- **Para maior assertividade, rodar com todos os dados de treino disponíveis**

In [90]:
# def objective_cv(trial):  
    
#     n_estimators = trial.suggest_int('n_estimators',20, 100)
#     eta = trial.suggest_float('eta',0.01, 0.05)
#     max_depth = trial.suggest_int('max_depth', 3, 9)
#     subsample = trial.suggest_float('subsample',0.1, 0.9)
#     colsample_bytree = trial.suggest_float('colsample_bytree',0.1,0.9)

#     # model definition
#     xgb_model = XGBRegressor( n_estimators    = n_estimators, 
#                               eta              = eta, 
#                               max_depth        = max_depth, 
#                               subsample        = subsample,
#                               colsample_bytree = colsample_bytree
#                              )
    
#     xgb_model.fit(X_train[features_selected], y_train)
    
#     # Model Prediction 
#     y_pred = xgb_model.predict(X_test_mod[features_selected])

#     # calculando rmse
#     smape = 100 / len(y_test_mod) * np.sum(2 * np.abs(np.expm1(y_pred) - np.expm1(y_test_mod)) / (np.abs(np.expm1(y_test_mod)) + np.abs(np.expm1(y_pred))))

#     return smape

In [91]:
# import optuna
# study = optuna.create_study(direction='minimize')
# study.optimize(objective_cv, n_trials=50)

In [92]:
# best_params = study.best_params
best_params = {'n_estimators': 100,
 'eta': 0.047830364071577154,
 'max_depth': 5,
 'subsample': 0.8457014395880995,
 'colsample_bytree': 0.17415570173518335}

In [93]:
best_model = XGBRegressor(n_jobs = -1 , 
                                 random_state=0,
                                 n_estimators = best_params['n_estimators'],
                                 eta = best_params['eta'],
                                 max_depth = best_params['max_depth'],
                                 subsample = best_params['subsample'],
                                 colsample_bytree = best_params['colsample_bytree'])

# model fit
best_model.fit(X_train[features_selected], y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.17415570173518335, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eta=0.047830364071577154, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=-1,
             num_parallel_tree=None, ...)

## Submission

### Carregar os dados

In [94]:
# Opening JSON file
sub = open('dataset/raw/test.json')
 
# returns JSON object as 
# a dictionary
data_sub_aux = json.load(sub)

data_sub = pd.json_normalize(data_sub_aux, record_path = 'data')
data_sub.columns = data_sub_aux['columns']
data_sub.shape

(5600, 16)

### Transformar os dados

In [95]:
def submission_preparation(df, features_selected):
    
    df = limpeza(df)
    df = feature_engen(df)
    df = rescalling(df)
    df = transform(df)
    
    return df[features_selected]

### Aplicar sobre os dados de teste

In [96]:
data_sub_mod = submission_preparation(data_sub, features_selected)

In [97]:
y_hat = np.expm1(best_model.predict(data_sub_mod))

In [98]:
df_submission = pd.DataFrame()
df_submission['pid'] = data_sub['pid']
df_submission['actual_price'] = y_hat

In [99]:
df_submission.to_csv('submissions/submission_12.csv', index=False)

------
# 10 Help Functions

In [100]:
def f(x):
    try:
        return ast.literal_eval(str(x))   
    except Exception as e:
        print(e)
        return []

In [101]:
def junta_dict(dict_list):
    dicionario = {}
    for d in dict_list:
        for key, value in d.items():
            dicionario[key] = value
    return dicionario

In [102]:
def feature_engen(df):
    ## out_of_stock - transformar para int 
    df['out_of_stock'] = df['out_of_stock'].astype('int64')

    # brand - substituir por outros
    df['brand'] = df['brand'].apply(lambda x: x.lower())

    # criando feature product
    df['product'] = df[['category', 'sub_category']].apply(lambda x: x['category'] + '_' + x['sub_category'], axis = 1)
    
    ## Tratando variaveis respostas nulas
    prices = df[['seller','pid', 'actual_price']].groupby('pid').max().reset_index()
    precos_nulos = df.loc[df['actual_price'].isna(), ['_id','pid']]
    precos_recuperados = pd.merge(precos_nulos, prices, how = 'inner', on = 'pid')
    
    df_aux = pd.merge(df, precos_recuperados[['_id', 'actual_price']], on = '_id', how = 'left')
    df['actual_price'] = df_aux['actual_price_x'].fillna(0) + df_aux['actual_price_y'].fillna(0)
    
    df = df.dropna(subset=['actual_price'])
    
    return df

In [103]:
def pipeline (df):
    df = limpeza(df)
    df = feature_engen(df)
    
    product_details = df['product_details'].apply(lambda x: f(x))
    product_details = pd.DataFrame([junta_dict(row) for row in product_details])
    
    return df, product_details


In [104]:
def test_preparation(df):
    X_test = rescalling(X_test)
    X_test = transform(X_test)
    y_test = log_var_resposta(y_test)
    
    return X_test